In [146]:
import pandas as pd
import numpy as np


In [147]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test = df_test.rename(columns={"num       ":"num"})
list_df = [df, df_test]

In [148]:
df_combine = df.append(df_test)
df_combine.reset_index(drop=True, inplace=True)

In [149]:
print(df_combine.shape)
df_combine = df_combine[df_combine.chol != '?']
df_combine = df_combine[df_combine.exang != '?']

df_combine.isin(['?']).sum(axis=0)

(1289, 14)


age            0
sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope       1177
ca          1276
thal        1251
num            0
dtype: int64

In [150]:
df.isin(['?']).sum(axis=0)

age            0
sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope       1129
ca          1197
thal        1181
num            0
dtype: int64

In [151]:
y = df['num']
X = df.drop(columns=['slope', 'ca', 'thal', 'num'])

y_combine = df_combine['num']
X_combine = df_combine.drop(columns=['slope', 'ca', 'thal', 'num'])

In [154]:
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=0)
# X_train, y_train = ros.fit_resample(X, y)
# print(pd.Series(y_train).value_counts())

# from imblearn.under_sampling import ClusterCentroids
# cc = ClusterCentroids(random_state=0)
# X_train, y_train = cc.fit_resample(X, y)
# print(pd.Series(y_train).value_counts())

# from imblearn.combine import SMOTEENN
# smote_enn = SMOTEENN(random_state=0)
# X_train, y_train = smote_enn.fit_resample(X, y)
# print(pd.Series(y_train).value_counts())


from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_train, y_train = smote_tomek.fit_resample(X, y)
print(pd.Series(y_train).value_counts())


1    998
0    998
dtype: int64


In [155]:
# X_train = X.values
# y_train = y.values

X_train_combine = X_combine.values
y_train_combine = y_combine.values

In [156]:
from scipy.stats import mode
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score

def purity_score(y_truth, y_predicted):
    labels = np.zeros_like(y_predicted)
    for i in range(2):
        mask = (y_predicted == i)
        labels[mask] = mode(y_truth[mask])[0]
    return accuracy_score(y_truth, labels)

## KMEANS

In [157]:
from sklearn.cluster import KMeans

In [158]:
kmeans = KMeans(2, random_state=1)
kmeans_combine = KMeans(2, random_state=1)

In [159]:
kmeans.fit(X_train)
kmeans_combine.fit(X_train_combine)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [163]:
print("Predicted label Train model only : ", kmeans.labels_)
print("Purity Score Train model only : ", recall_score(y_train, kmeans.labels_, average=None))

print()

print("Predicted label Combine model : ", kmeans_combine.labels_)
print("Purity Score Combine model : ", purity_score(y_train_combine, kmeans_combine.labels_))

Predicted label Train model only :  [1 1 1 ... 1 0 0]
Purity Score Train model only :  [0.10721443 0.83466934]

Predicted label Combine model :  [0 0 0 ... 0 0 0]
Purity Score Combine model :  0.8232994526974199


## Agglomerative

In [164]:
from sklearn.cluster import AgglomerativeClustering

In [165]:
agg = AgglomerativeClustering(2, linkage="ward")
agg.fit(X_train)

agg_combine = AgglomerativeClustering(2, linkage="ward")
agg_combine.fit(X_train_combine)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2,
            pooling_func='deprecated')

In [170]:
print("Predicted label Train model only : ", agg.labels_)
print("Purity Score Train model only : ", recall_score(y_train, agg.labels_, average=None))

print()

print("Predicted label Combine model : ", agg_combine.labels_)
print("Purity Score Combine model : ", purity_score(y_train_combine, agg_combine.labels_))

Predicted label Train model only :  [1 1 0 ... 1 0 0]
Purity Score Train model only :  [0.55811623 0.35170341]

Predicted label Combine model :  [1 1 1 ... 1 1 1]
Purity Score Combine model :  0.8232994526974199


## DBSCAN

In [129]:
from sklearn.cluster import DBSCAN

In [130]:
dbscan_model = DBSCAN(min_samples=2, eps=0.5)
dbscan_model.fit(X_train)

dbscan_model_combine = DBSCAN(min_samples=2, eps=0.5)
dbscan_model_combine.fit(X_train_combine)

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)

In [131]:
print("Predicted label Train model only : ", dbscan_model.labels_)
print("Purity Score Train model only : ", purity_score(y_train, dbscan_model.labels_))

print()

print("Predicted label Combine model : ", dbscan_model_combine.labels_)
print("Purity Score Combine model : ", purity_score(y_train_combine, dbscan_model_combine.labels_))

Predicted label Train model only :  [-1 -1 -1 ... -1 -1 96]
Purity Score Train model only :  0.8333333333333334

Predicted label Combine model :  [-1 -1 -1 ... -1 -1 -1]
Purity Score Combine model :  0.8232994526974199
